<a href="https://colab.research.google.com/github/Javigomez4/my-second-repo/blob/main/Problem_4d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =====================================
# Problem 4 – Quadratic 0-1 linearized
# MILP and LP relaxation
# =====================================

!pip install pulp

import pulp

# -------------------------------
# Example data (replace with yours)
# -------------------------------

n = 5  # number of items (example)
K = 2  # cardinality bound (example)

# alpha_i coefficients (example)
alpha = {i: 1.0 + 0.1*i for i in range(n)}  # e.g. [1.0, 1.1, 1.2, 1.3, 1.4]

# w_ij coefficients (example symmetric weights)
w = {}
for i in range(n):
    for j in range(i+1, n):
        w[(i, j)] = 0.5  # example constant weight; replace with your data

# =====================================================
# 1) MILP model (X binary, Y binary or continuous [0,1])
# =====================================================

milp = pulp.LpProblem("Quadratic_Linearized_MILP", pulp.LpMaximize)

# Decision variables
X = {i: pulp.LpVariable(f"X_{i}", lowBound=0, upBound=1, cat="Binary") for i in range(n)}

# Y_ij for i<j, can be Binary or Continuous(0,1)
Y = {}
for i in range(n):
    for j in range(i+1, n):
        Y[(i, j)] = pulp.LpVariable(f"Y_{i}_{j}", lowBound=0, upBound=1, cat="Binary")

# Objective: sum alpha_i X_i + sum w_ij Y_ij
milp += (
    pulp.lpSum(alpha[i] * X[i] for i in range(n))
    + pulp.lpSum(w[(i,j)] * Y[(i,j)] for i in range(n) for j in range(i+1, n))
)

# Constraint: sum X_i <= K  (example)
milp += pulp.lpSum(X[i] for i in range(n)) <= K, "Cardinality"

# Linearization constraints: for all i<j
for i in range(n):
    for j in range(i+1, n):
        milp += Y[(i, j)] <= X[i],        f"Y_le_Xi_{i}_{j}"
        milp += Y[(i, j)] <= X[j],        f"Y_le_Xj_{i}_{j}"
        milp += Y[(i, j)] >= X[i] + X[j] - 1, f"Y_ge_Xi_plus_Xj_minus1_{i}_{j}"

# Solve MILP
milp.solve(pulp.PULP_CBC_CMD(msg=False))

print("=== MILP solution ===")
print("Status:", pulp.LpStatus[milp.status])
print("Objective value (MILP):", pulp.value(milp.objective))
print("X* =", {i: X[i].varValue for i in range(n)})
print("Y* =", {(i,j): Y[(i,j)].varValue for i in range(n) for j in range(i+1, n)})

# ======================================
# 2) LP relaxation (X, Y continuous)
# ======================================

lp_relax = pulp.LpProblem("Quadratic_Linearized_LP_Relax", pulp.LpMaximize)

# Continuous X and Y in [0,1]
X_LP = {i: pulp.LpVariable(f"X_{i}", lowBound=0, upBound=1, cat="Continuous") for i in range(n)}
Y_LP = {}
for i in range(n):
    for j in range(i+1, n):
        Y_LP[(i, j)] = pulp.LpVariable(f"Y_{i}_{j}", lowBound=0, upBound=1, cat="Continuous")

# Objective
lp_relax += (
    pulp.lpSum(alpha[i] * X_LP[i] for i in range(n))
    + pulp.lpSum(w[(i,j)] * Y_LP[(i,j)] for i in range(n) for j in range(i+1, n))
)

# Same constraints: sum X_i <= K
lp_relax += pulp.lpSum(X_LP[i] for i in range(n)) <= K, "Cardinality"

# Same linearization constraints
for i in range(n):
    for j in range(i+1, n):
        lp_relax += Y_LP[(i, j)] <= X_LP[i],        f"Y_LP_le_Xi_{i}_{j}"
        lp_relax += Y_LP[(i, j)] <= X_LP[j],        f"Y_LP_le_Xj_{i}_{j}"
        lp_relax += Y_LP[(i, j)] >= X_LP[i] + X_LP[j] - 1, f"Y_LP_ge_Xi_plus_Xj_minus1_{i}_{j}"

# Solve LP relaxation
lp_relax.solve(pulp.PULP_CBC_CMD(msg=False))

print("\n=== LP RELAXATION solution ===")
print("Status:", pulp.LpStatus[lp_relax.status])
print("Objective value (LP):", pulp.value(lp_relax.objective))
print("X_LP* =", {i: X_LP[i].varValue for i in range(n)})
print("Y_LP* =", {(i,j): Y_LP[(i,j)].varValue for i in range(n) for j in range(i+1, n)})


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 41.8 MB/s eta 0:00:00
=== MILP solution ===
Status: Optimal
Objective value (MILP): 3.2
X* = {0: 0.0, 1: 0.0, 2: 0.0, 3: 1.0, 4: 1.0}
Y* = {(0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (0, 4): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 0.0, (2, 3): 0.0, (2, 4): 0.0, (3, 4): 1.0}

=== LP RELAXATION solution ===
Status: Optimal
Objective value (LP): 4.400000000000001
X_LP* = {0: 0.4, 1: 0.4, 2: 0.4, 3: 0.4, 4: 0.4}
Y_LP* = {(0, 1): 0.4, (0, 2): 0.4, (0, 3): 0.4, (0, 4): 0.4, (1, 2): 0.4, (1, 3): 0.4, (1, 4): 0.4, (2, 3): 0.4, (2, 4): 0.4, (3, 4): 0.4}
